## Check List 4 - MotionFilter
* Motion filters are quick decision makers that prevents trying to plan a infeasible motion.
* Here, ReachChecker is tested, compared with actual planning results in random trials
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4-A.2 ReachChecker**  
  - pick, place, ~~sweep~~
    - single process
    - multi process
    
* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
tip_link = crob.get_robot_tip_dict()[crob.robot_names[0]]
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries - set collision=False for all, to make reach-only problems

In [3]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=False)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=False)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepTool

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name=tip_link, 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name=tip_link,
                              dims=(0.02,0.02,0.1), center=(0,0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name=tip_link, 
                              dims=(0.02,0.02,0.1), center=(0,-0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name=tip_link, 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepTool, point=(0,0,0), rpy=(0,0,0))

##### create_subject

In [7]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepPoint
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

##### prepare motion planner

In [8]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)

##### ReachChecker
- pick, place, ~~sweep~~

In [9]:
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.constraint.constraint_common import calc_redundancy

In [10]:
# ReachChecker needs end_link_couple_dict, which is a dictionary of list of coupled link names
mfilter = ReachChecker(pscene)

##### pick & place scenarios
  - below cell will automatically generate random configurations and record the filtering algorithm and planning results

##### Test a few runs with visualization

In [11]:
VISUALIZE = True
result_dict = defaultdict(list)
N_TRIAL = 10

for i_count in range(N_TRIAL):
    box1.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    goal.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    scenario = "pick"
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([('box1', 'bottom', 'floor', 'floor')], crob.home_pose)
    
    obj_name, handle_name, actor_name, actor_geo = "box1", "handle1", "grip0", "gripper"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=initial_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        pick_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    obj_name, handle_name, actor_name, actor_geo = "box1", "bottom", "goal", "goal"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=pick_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(crob.home_pose)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
0/10: Result match for pick: True / True
try transition motion
transition motion tried: False
0/10: Result match for place: False / False
try transition motion
transition motion tried: False
1/10: Result match for pick: False / False
try transition motion
transition motion tried: True
2/10: Result match for pick: True / True
try transition motion
transition motion tried: False
2/10: Result match for place: False / False
try transition motion
transition motion tried: True
3/10: Result match for pick: True / True
try transition motion
transition motion tried: False
3/10: Result match for place: False / False
try transition motion
transition motion tried: True
4/10: Result match for pick: True / True
try transition motion
transition motion tried: False
4/10: Result match for place: False / False
try transition motion
transition motion tried: False
5/10: Result match for pick: False / False
try transition motion
transition motion tried: F

##### massively test without visualization

In [ ]:
VISUALIZE = False
result_dict = defaultdict(list)
N_TRIAL = 1000

for i_count in range(N_TRIAL):
    box1.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    goal.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    scenario = "pick"
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([('box1', 'bottom', 'floor', 'floor')], crob.home_pose)
    
    obj_name, handle_name, actor_name, actor_geo = "box1", "handle1", "grip0", "gripper"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=initial_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        pick_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    obj_name, handle_name, actor_name, actor_geo = "box1", "bottom", "goal", "goal"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False
    from_state=pick_state
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        ((obj_name, handle_name, actor_name, actor_geo), 
         to_state.binding_state[1]), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(crob.home_pose)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

In [ ]:
ACCURACY_CUT = 95
print("Accuracy cutline: {} %".format(ACCURACY_CUT))
for k, v in result_dict.items():
    v_arr = np.array(v)
    suc_idx = np.where(v_arr[:, 1])[0]
    acc_val = np.round(np.mean(v_arr[:,0]==v_arr[:,1])*100, 1)
    acc_suc = np.round(np.mean(v_arr[suc_idx,0]==v_arr[suc_idx,1])*100, 1)
    print("="*60)
    print("{} accuracy: {} % / success accuracy: {} % / success count:{}/{}".format(k, acc_val, acc_suc, len(suc_idx), len(v_arr)))
    assert acc_val>ACCURACY_CUT and acc_suc>ACCURACY_CUT, \
            "Accuracy {}/{} is lower than {}. Consider debugging the algorithm.".format(acc_val, acc_suc, ACCURACY_CUT)
    print("""OK with "{}" scenario""".format(k))

##### sweep scenarios: reaching first sweep point and sweeping to second waypoint
  - as constrained motion is not considered in ReachChecker, we remove constraint of SweepLineTask here

In [12]:
# remove constraints here
sweep.make_constraints = lambda binding_from, binding_to: []

##### test a few runs with visualization

In [13]:
VISUALIZE = True
result_dict = defaultdict(list)
N_TRIAL = 10

for i_count in range(N_TRIAL):
    floor.set_offset_tf(center=(np.random.rand(3)-0.5)*1, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*pi/2))
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([('box1', 'bottom', 'floor', 'floor')], crob.home_pose)

    scenario = "reach wp1"
    obj_name, handle_name, actor_name, actor_geo = "sweep", "wp1", "sweep_face", "sweep_face"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False

    from_state=initial_state.copy(pscene)
    from_state.Q = crob.home_pose
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        (to_state.binding_state[1], 
         (obj_name, handle_name, actor_name, actor_geo)), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
            
    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        sweep1_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
#     assert success_mfilter == success, "Failure: filter result not same with motion planning result"

    if not success:
        pscene.set_object_state(initial_state)
        continue
        
    scenario = "sweep to wp2"
    obj_name, handle_name, actor_name, actor_geo = "sweep", "wp2", "sweep_face", "sweep_face"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = True     # by setting constrained = True, although no effect in ReachChecker

    from_state=sweep1_state.copy(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        (to_state.binding_state[1], 
         (obj_name, handle_name, actor_name, actor_geo)), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values,
        timeout=1)
    
    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
#     assert success_mfilter == success, "Failure: filter result not same with motion planning result"

try transition motion
transition motion tried: True
0/10: Result match for reach wp1: True / True
try transition motion
transition motion tried: False
0/10: Result match for sweep to wp2: False / False
try transition motion
transition motion tried: True
1/10: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
1/10: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
2/10: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
2/10: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: False
3/10: Result match for reach wp1: False / False
try transition motion
transition motion tried: True
4/10: Result match for reach wp1: True / True
try transition motion
transition motion tried: True
4/10: Result match for sweep to wp2: True / True
try transition motion
transition motion tried: True
5/10: Result match for reach wp1: True / True
tr

##### massively test without visualization

In [ ]:
VISUALIZE = False
result_dict = defaultdict(list)
N_TRIAL = 1000

for i_count in range(N_TRIAL):
    floor.set_offset_tf(center=(np.random.rand(3)-0.5)*1, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*pi/2))
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([('box1', 'bottom', 'floor', 'floor')], crob.home_pose)

    scenario = "reach wp1"
    obj_name, handle_name, actor_name, actor_geo = "sweep", "wp1", "sweep_face", "sweep_face"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = False

    from_state=initial_state.copy(pscene)
    from_state.Q = crob.home_pose
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        (to_state.binding_state[1], 
         (obj_name, handle_name, actor_name, actor_geo)), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values)
            
    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        sweep1_state = pscene.rebind_all(binding_list, LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
#     assert success_mfilter == success, "Failure: filter result not same with motion planning result"

    if not success:
        pscene.set_object_state(initial_state)
        continue
        
    scenario = "sweep to wp2"
    obj_name, handle_name, actor_name, actor_geo = "sweep", "wp2", "sweep_face", "sweep_face"
    actor = pscene.actor_dict[actor_name]
    obj = pscene.subject_dict[obj_name]
    handle = obj.action_points_dict[handle_name]
    binding_list = [(obj_name, handle_name, actor_name, actor_geo)]
    constrained = True     # by setting constrained = True, although no effect in ReachChecker

    from_state=sweep1_state.copy(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    to_state = from_state.copy(pscene)
    to_state.set_binding_state(
        (to_state.binding_state[1], 
         (obj_name, handle_name, actor_name, actor_geo)), 
        pscene)
    redundancy_dict = {
        obj_name:{handle_name:{"w":np.random.rand()*2*np.pi}, 
                  actor_name:{"w":np.random.rand()*2*np.pi}}}
    redundancy_values = {(obj_name, handle.name): calc_redundancy(redundancy_dict[obj_name][handle_name], handle),
                         (obj_name, actor.name): calc_redundancy(redundancy_dict[obj_name][actor_name], actor)
                        }

    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    success_mfilter = mfilter.check(actor, obj, handle, redundancy_values, Q_dict,constrained)
    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, binding_list=binding_list, redundancy_values=redundancy_values,
        timeout=1)
    
    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
#     assert success_mfilter == success, "Failure: filter result not same with motion planning result"

In [ ]:
ACCURACY_CUT = 95
print("Accuracy cutline: {} %".format(ACCURACY_CUT))
for k, v in result_dict.items():
    v_arr = np.array(v)
    suc_idx = np.where(v_arr[:, 1])[0]
    acc_val = np.round(np.mean(v_arr[:,0]==v_arr[:,1])*100, 1)
    acc_suc = np.round(np.mean(v_arr[suc_idx,0]==v_arr[suc_idx,1])*100, 1)
    print("{} accuracy: {} % / success accuracy: {} % / success count:{}/{}".format(k, acc_val, acc_suc, len(suc_idx), len(v_arr)))
    assert acc_val>ACCURACY_CUT and acc_suc>ACCURACY_CUT, \
            "Accuracy {}/{} is lower than {}. Consider debugging the algorithm.".format(acc_val, acc_suc, ACCURACY_CUT)
    print("""OK with "{}" scenario""".format(k))

##### planning: pick & place & sweep
- *NOTE*: We disabled all collision and motion constraint to check ReachChecker's functionality only.

In [14]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene, 
                                        motion_filters=[ReachChecker(pscene)]))
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene))

###### reset object positions

In [17]:
floor.set_offset_tf(center=(0,0,0), orientation_mat=Rot_rpy((0,0,0)))
goal.set_offset_tf(center=(0.3,-0.4,0), orientation_mat=Rot_rpy((0,0,0)))
box1.set_offset_tf(center=(0.3,0.4,0.031), orientation_mat=Rot_rpy((0,0,0)))
gscene.update_markers_all()

initial_state = pscene.initialize_state(crob.home_pose)
pscene.set_object_state(initial_state)

##### - single process

In [18]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (0.02/100.0 s, steps/err: 13(17.0531272888 ms)/0.00169053380359)
try: 1 - ('floor', 1)->('floor', 2)
try transition motion
transition motion tried: True
result: 1 - ('floor', 1)->('floor', 2) = success
branching: 1->2 (0.04/100.0 s, steps/err: 8(18.1918144226 ms)/0.00203393582438)
try: 2 - ('floor', 2)->('gripper', 2)
try transition motion
transition motion tried: True
=============== try reaching goal from ('gripper', 2) =================
result: 2 - ('floor', 2)->('gripper', 2) = success
branching: 2->3 (0.06/100.0 s, steps/err: 23(18.6247825623 ms)/0.00218810680239)
try: 3 - ('gripper', 2)->('goal', 2)
try transition motion
transition motion tried: True
result: 3 - ('gripper', 2)->('goal', 2) = success
branching: 3->4 (0.07/100.0 s, steps/err: 8(17.7659988403 ms)/0.00103557312606)
++ adding return motion to acquired answer ++
try jo

##### - multi process

In [19]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

Use 20/20 agents
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
result: 0 - ('floor', 0)->('gripper', 0) = fail
transition motion tried: True
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->1 (0.11/100.0 s, steps/err: 17(60.8861446381 ms)/0.0015204366685)
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
try: 1 - ('gripper', 0)->('floor', 0)
branching: 0->2 (0.12/100.0 s, steps/err: 7(60.555934906 ms)/0.00174682084787)
result: 1 - ('gripper', 0)->('floor', 0) = fail
try: 1 - ('gripper', 0)->('gripper', 1)
try transition motion
try: 1 - ('gripper', 0)->('gripper', 1)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
transition motion tried: True
transition motion tried: True
try transition motion
result: 1 - ('gripper', 0)->('gripper', 1) = s

result: 1 - ('gripper', 0)->('gripper', 1) = success
try transition motion
branching: 1->19 (0.52/100.0 s, steps/err: 31(74.8658180237 ms)/0.00148977574976)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
result: 12 - ('gripper', 2)->('goal', 2) = success
transition motion tried: True
transition motion tried: True
branching: 12->18 (0.53/100.0 s, steps/err: 49(97.6870059967 ms)/0.00191302344367)
transition motion tried: True
try transition motion
try transition motion
result: 2 - ('floor', 1)->('floor', 2) = success
try: 11 - ('gripper', 2)->('floor', 2)
result: 11 - ('gripper', 2)->('floor', 2) = fail
try: 20 - ('floor', 2)->('gripper', 2)
branching: 2->20 (0.55/100.0 s, steps/err: 8(90.3499126434 ms)/0.00172432802019)
try: 9 - ('floor', 1)->('floor', 2)
transition motion tried: True
result: 10 - ('gripper', 0)->('goal', 0) = success
++ adding return motion to acquired answer ++
result: 1 - ('gripper', 0)->('floor', 0) = fail
branching: 10->21 (0.58/100.0 s, steps/err: 18(12

transition motion tried: True
try: 39 - ('goal', 1)->('goal', 2)
=============== try reaching goal from ('gripper', 2) =================
transition motion tried: True
joint motion tried: True
=============== try reaching goal from ('goal', 1) =================
try transition motion
result: 10 - ('gripper', 0)->('gripper', 1) = success
result: 34 - ('floor', 2)->('gripper', 2) = success
try transition motion
branching: 10->43 (0.85/100.0 s, steps/err: 22(64.7859573364 ms)/0.00181745207536)
=============== try reaching goal from ('goal', 1) =================
try: 40 - ('goal', 1)->('goal', 2)
++ adding return motion to acquired answer ++
result: 17 - ('floor', 0)->('floor', 1) = success
branching: 34->41 (0.85/100.0 s, steps/err: 3(61.5108013153 ms)/0.00103618491778)
transition motion tried: True
joint motion tried: True
branching: 17->42 (0.86/100.0 s, steps/err: 32(139.721155167 ms)/0.000745023535871)
result: 14 - ('goal', 0)->('goal', 1) = success
=============== try reaching goal fro

=============== try reaching goal from ('gripper', 2) =================
++ adding return motion to acquired answer ++
======================= terminated 14: first answer acquired from other agent ===============================
result: 3 - ('gripper', 1)->('gripper', 2) = success
try joint motion
result: 41 - ('gripper', 2)->('goal', 2) = success
branching: 3->55 (0.99/100.0 s, steps/err: 30(133.350133896 ms)/0.00162615302302)
joint motion tried: True
branching: 41->56 (0.99/100.0 s, steps/err: 25(68.8879489899 ms)/0.00135722082434)
++ adding return motion to acquired answer ++
======================= terminated 4: first answer acquired from other agent ===============================
try joint motion
joint motion tried: True
joint motion tried: True
======================= terminated 18: first answer acquired ===============================
======================= terminated 17: first answer acquired ===============================
======================= terminated 2: first answer ac

Process Process-18:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 152, in __search_loop
  File "pkg/planning/pipeline.py", line 152, in __search_loop
    display=display, dt_vis=dt_vis, **kwargs)
  File "pkg/planning/pipeline.py", line 200, in test_connection
    display=display, dt_vis=dt_vis, **kwargs)
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/pipeline.py", line 200, in test_connection
  File "pkg/planning/mo